In [10]:
from chessdotcom import get_leaderboards, get_player_stats, get_player_game_archives, get_titled_players, get_player_profile
import pprint
import requests
import pandas as pd
import io
import chess
import warnings
warnings.filterwarnings('ignore')
printer = pprint.PrettyPrinter()

In [7]:
def get_player_info(player_id):
    #get best ranking
    stats = get_player_stats(player_id).json
    best_ranking = stats['stats']['chess_rapid']['best']['rating']
    #get country
    profile = get_player_profile(player_id).json
    country_url = profile['player']['country']
    country = requests.get(country_url).json()['name']
    #get title
    if 'title' in profile['player'].keys():
        title = profile['player']['title']
    else:
        title = "none"
    #get first_month and last_month of activity
    games = get_player_game_archives(player_id,0).json
    first_game_url = games['archives'][0]
    last_game_url = games['archives'][-1]
    first_month = chess.pgn.read_game(io.StringIO(requests.get(first_game_url).json()['games'][0]['pgn'])).headers['Date']
    last_month = chess.pgn.read_game(io.StringIO(requests.get(last_game_url).json()['games'][0]['pgn'])).headers['Date']
    
    return(best_ranking, first_month, last_month, country, title)
    

In [87]:
my_data = pd.DataFrame(columns = ['player_id','color','opponent','date','player_ranking','game_type','outcome','pgn'])

In [92]:
my_data = pd.DataFrame(columns = ['player_id','color','opponent','date','player_ranking','game_type','outcome','pgn'])
games_list = []
my_games = get_player_game_archives('baudouindupont',0).json
for i in range(len(my_games['archives'])):
    game = requests.get(my_games['archives'][i]).json()
    games_list.append(game)


In [8]:
def get_color(player_id,pgn):
    header = dict(pgn.headers.items())
    if header['White']==player_id:
        return 'White'
    else :
        return 'Black'
    
def get_games(player_id,months):
    games_list = []
    my_games = get_player_game_archives(player_id,0).json
    for i in range(len(my_games['archives'])):
        game = requests.get(my_games['archives'][i]).json()
        games_list.append(game)

    pgn_list = []
    for i in range(len(games_list)):
        for j in range(len(games_list[i]['games'])):
            if 'pgn' in games_list[i]['games'][j] : 
                pgn = chess.pgn.read_game(io.StringIO(games_list[i]['games'][j]['pgn']))
                pgn_list.append(pgn)
            else:
                continue
    #set final data frame
    data = pd.DataFrame(columns = ['date','time','game_type','player_color','player_id','player_ranking','opponent','opponent_ranking','ECO','outcome','game'])
    #create row to append
    for k in range(len(pgn_list)):
        headers = dict(pgn_list[k].headers.items())
        date = headers['UTCDate']
        time = headers['UTCTime']
        game_type = headers['TimeControl']
        player_color = get_color(player_id,pgn_list[k])
        player_ranking = headers['WhiteElo'] if player_color == 'White' else headers['BlackElo']
        opponent = headers['White'] if player_color == 'Black' else headers['Black']
        opponent_ranking = headers['WhiteElo'] if player_color == 'Black' else headers['BlackElo']
        ECO = headers['ECO'] if 'ECO' in headers.keys() else 'none'
        outcome = headers['Termination']
        game = str(pgn_list[k].mainline_moves())
        new_row = [date,time,game_type,player_color,player_id,player_ranking,opponent,opponent_ranking,ECO,outcome,game]
        data = data.append(pd.Series(new_row, index=data.columns), ignore_index=True)
    return data
    
         

In [ ]:
gm_list = get_titled_players('GM',0).json['players']

data = pd.DataFrame(columns = ['date','time','game_type','player_color','player_id','player_ranking','opponent','opponent_ranking','ECO','outcome','game'])

for i in range(len(gm_list)):
    data = pd.concat([data,get_games(str(gm_list[i]))])

In [177]:
data.to_csv('/Users/Baudouin/Desktop/chess.csv')

In [153]:
gm_list
a = get_player_game_archives(gm_list[0],0).json

In [176]:
data

,date,time,game_type,player_color,player_id,player_ranking,opponent,opponent_ranking,ECO,outcome,game
0,2014.09.07,03:12:32,120+1,White,123lt,1343,kingbos,1129,D08,123lt won by resignation,1. d4 { [%clk 0:02:00] } 1... d5 { [%clk 0:02:...
1,2014.09.07,03:15:33,120+1,Black,123lt,1469,betablokator,1266,B21,123lt won by resignation,1. e4 { [%clk 0:02:00] } 1... c5 { [%clk 0:02:...
2,2014.09.07,03:19:54,120+1,Black,123lt,1592,Computer3-HARD,1529,D10,123lt won by checkmate,1. d4 { [%clk 0:02:00] } 1... d5 { [%clk 0:02:...
3,2014.09.07,03:23:33,120+1,White,123lt,1577,Computer3-HARD,1521,C41,Game drawn by insufficient material,1. e4 { [%clk 0:02:00] } 1... e5 { [%clk 0:02:...
4,2014.09.07,03:26:35,120+1,Black,123lt,1491,Computer3-HARD,1528,D00,Computer3-HARD won by resignation,1. d4 { [%clk 0:02:00] } 1... d5 { [%clk 0:02:...
...,...,...,...,...,...,...,...,...,...,...,...
404,2021.11.23,19:09:03,180+1,Black,alexsur1981,3067,Alexsur1981,2844,B23,Game drawn by repetition,1. e4 { [%clk 0:03:00.9] } 1... c5 { [%clk 0:0...
405,2021.11.23,19:19:50,180+1,Black,alexsur1981,2853,SF-96,2804,A07,Alexsur1981 won by resignation,1. Nf3 { [%clk 0:03:00.9] } 1... d5 { [%clk 0:...
406,2021.11.23,19:36:32,180+1,Black,alexsur1981,3002,Alexsur1981,2847,B23,wonderfultime won on time,1. e4 { [%clk 0:02:58.9] } 1... c5 { [%clk 0:0...
407,2021.11.23,19:47:36,180+1,Black,alexsur1981,2859,Chopper1905,2948,A25,Alexsur1981 won on time,1. c4 { [%clk 0:03:00.9] } 1... e5 { [%clk 0:0...


In [155]:
game = requests.get(a['archives'][-3]).json()
game

{'games': [{'url': 'https://www.chess.com/game/live/34762832595',
   'pgn': '[Event "Live Chess"]\n[Site "Chess.com"]\n[Date "2022.01.01"]\n[Round "-"]\n[White "123lt"]\n[Black "Juwen"]\n[Result "1/2-1/2"]\n[CurrentPosition "8/R7/pr4kp/8/1P6/6K1/7P/8 b - -"]\n[Timezone "UTC"]\n[ECO "E02"]\n[ECOUrl "https://www.chess.com/openings/Catalan-Opening-Open-Defense...6.Qxc4-c5-7.Nf3-a6"]\n[UTCDate "2022.01.01"]\n[UTCTime "08:00:54"]\n[WhiteElo "2506"]\n[BlackElo "2515"]\n[TimeControl "900+5"]\n[Termination "Game drawn by agreement"]\n[StartTime "08:00:54"]\n[EndDate "2022.01.01"]\n[EndTime "08:39:26"]\n[Link "https://www.chess.com/game/live/34762832595"]\n\n1. d4 {[%clk 0:14:55.8]} 1... Nf6 {[%clk 0:15:00.6]} 2. c4 {[%clk 0:14:59.1]} 2... e6 {[%clk 0:15:04.8]} 3. g3 {[%clk 0:15:02.6]} 3... d5 {[%clk 0:15:05.9]} 4. Bg2 {[%clk 0:15:05.6]} 4... dxc4 {[%clk 0:14:43]} 5. Qa4+ {[%clk 0:15:07.7]} 5... Nbd7 {[%clk 0:14:39.4]} 6. Qxc4 {[%clk 0:15:10.4]} 6... c5 {[%clk 0:14:41.4]} 7. Nf3 {[%clk 0:15:09.

In [9]:
a = get_games('baudouindupont')

AttributeError: module 'chess' has no attribute 'pgn'